In [1]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import pandas as pd
from pandas import DataFrame
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score
import sys
import os
import mlflow
from datetime import datetime

/Users/nikitalukasevic/Desktop/Work/Projects/classifier-nikita/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#change path to the root directory
%cd /Users/nikitalukasevic/Desktop/Work/Projects/classifier-nikita/

/Users/nikitalukasevic/Desktop/Work/Projects/classifier-nikita


/Users/nikitalukasevic/Desktop/Work/Projects/classifier-nikita/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
#use ! instead of % to run on google colab
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Add src as folder from where to import
parent_dir = os.path.abspath(os.getcwd())

# Add this directory to sys.path
sys.path.append(parent_dir)
print(parent_dir)

/Users/nikitalukasevic/Desktop/Work/Projects/classifier-nikita


In [5]:
#this code uploads the credentials to MLflow given in the .env file
from src.settings import (
    MLFLOW_TRACKING_URI,
    MLFLOW_TRACKING_USERNAME,
    MLFLOW_TRACKING_PASSWORD,
)
print(MLFLOW_TRACKING_USERNAME)

Nikita


In [6]:
# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model_path = 'model_2024_05_30_1047_50.pt' #path to the weights
model.load_state_dict(torch.load(model_path))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: 'model_2024_05_30_1047_50.pt'

In [ ]:
columns_to_extract: list[str] = ['text', 'text_b', 'label']

#delete nrows parameter if device = 'cpu'
test_df: DataFrame = pd.read_csv('../data/work_data/test_work.csv', usecols=columns_to_extract)

In [ ]:
texts = test_df['text'].tolist()
texts_b = test_df['text_b'].tolist()
labels = test_df['label'].tolist()

In [ ]:
inputs = tokenizer(texts, texts_b, return_tensors='pt', max_length=512, truncation=True, padding=True)

In [ ]:
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

In [ ]:
user = MLFLOW_TRACKING_USERNAME

experiment_name = 'Nikita_PatentMatchBaseline'


def timestamp():
    """This function creates current timestamp"""
    return datetime.now().strftime("%Y_%m_%d_%H%M_%S")

In [ ]:
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None or experiment.lifecycle_stage == 'deleted':
    mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment_by_name(experiment_name)

In [ ]:
def get_weights_version(weights_path: str) -> str:
    path_tolist = weights_path.split('/')
    return path_tolist[len(path_tolist)-1]

In [ ]:
with mlflow.start_run(experiment_id=experiment.experiment_id):
    mlflow.log_param('weight_version', get_weights_version(model_path))
    
    # set run name
    mlflow.set_tag(key='mlflow.runName',
                    value=f'eval_{timestamp()}_{user}')
    
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten()

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    val_accuracy = accuracy_score(true_labels, predictions)
    print(f"TEST Accuracy: {val_accuracy}")
    mlflow.log_metric("test_accuracy", val_accuracy)
    
    # Calculate and log the accuracy of the predictions
    val_f1 = f1_score(true_labels, predictions, average='weighted')
    print(f"TEST F1 Score: {val_f1}")
    mlflow.log_metric("val_f1", val_f1)
    
    # Calculate and log the accuracy of the predictions
    val_mcc = matthews_corrcoef(true_labels, predictions)
    print(f"TEST MCC: {val_mcc}")
    mlflow.log_metric("test_mcc", val_mcc)

    print(f'test_accuracy: {val_accuracy}')
    print(f'test_f1: {val_f1}')
    print(f'test_mcc: {val_mcc}')